Agents

Scratch -> LLM + functions inside its prompt
OpenAI Function calling -> using the official api to call functions for LLMs
Build simple agents with langchain

In [1]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a simple task list of 3 desktop things I can do on the terminal.")
Markdown(output)

1. Check network connectivity: Use the command "ping [website or IP address]" to check if your computer is able to connect to a specific website or IP address. For example, "ping www.google.com" will send network packets to Google's server and display the responsiveness of the connection.

2. View system information: Use the command "uname -a" to display detailed information about your system, including the kernel version, operating system, and hardware architecture. This can be useful for troubleshooting or obtaining information about your computer.

3. List directory contents: Use the command "ls" to display the contents of the current directory. This will show you all the files and subdirectories within the current location. You can also add options to customize the output, such as "ls -l" to display a long list format with additional details like file permissions and sizes.

Ok cool, so here we have three ideas of actions to perform:

Creating directories
Listing files
Removing files
Let's transform them into functions that we could call just like in any type of Python-based application.

In [1]:
import subprocess

def create_directory(directory_name):
    # Execute shell command to create directory
    subprocess.run(["mkdir", directory_name])

def create_file():
    subprocess.run(["touch", "test.txt"])

def list_files():
    subprocess.run(["ls"])

In [2]:
create_directory("test")

mkdir: cannot create directory ‘test’: File exists


OK, cool! now Notice that, here we added single parameters to the functions: create_directory(), create_file(), and we did this so that we can actually do real things instead of just always creating the same folders over and over.

Now, how can we actually put it all together so that given a task, a model can:

Plan the task
Execute actions to complete the task
Know when to call a function

This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [21]:
 class ModelWithTools:
    def __init__(self, model):
        self.model = model
    
    def get_response(self, prompt_question):
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                    {"role": "user", "content": prompt_question}]
        )
        
        return response.choices[0].message.content
    
    def create_directory(self, directory_name):
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        subprocess.run(["touch", file_name])
    
    def list_files(self):
        subprocess.run(["ls"])
    
    

model = ModelWithTools("gpt-3.5-turbo-16k")
task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder, create a file called 'the-10-master-rules.md"
output = model.get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:
                            
    def create_directory(self, directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(self, file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files(self):
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

To complete the task, you need to execute the following Python function calls:

create_directory('lucas-the-agent-master')
create_file('lucas-the-agent-master/the-10-master-rules.md')

Hey! Look at that the output is that function! Now, all we need is to direct this output to be executed somehow!

In [19]:
exec("model." + output)

TypeError: ModelWithTools.create_directory() takes 0 positional arguments but 2 were given